# Базовая классификация текстов

В этом руководстве демонстрируется классификация текста, начиная с простых текстовых файлов, хранящихся на диске. Вы обучите двоичный классификатор выполнять анализ настроений в наборе данных IMDB. В конце записной книжки вы можете попробовать выполнить упражнение, в котором вы обучите многоклассовый классификатор предсказывать тег для вопроса о программировании в Stack Overflow.

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses


In [ ]:
print(tf.__version__)

2.8.0


## Sentiment analysis

Это задание обучает модель анализа настроений классифицировать отзывы о фильмах как положительные или отрицательные на основе текста обзора. Это пример бинарной — или двухклассовой — классификации, важной и широко применимой проблемы машинного обучения.

Вы будете использовать большой набор данных обзоров фильмов , который содержит текст 50 000 обзоров фильмов из базы данных фильмов в Интернете . Они разделены на 25 000 отзывов для обучения и 25 000 отзывов для тестирования. Наборы для обучения и тестирования сбалансированы , то есть содержат равное количество положительных и отрицательных отзывов.


### Загрузите и изучите набор данных IMDB
Давайте загрузим и извлечем набор данных, а затем изучим структуру каталогов.

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb_v1')

84140032/84125825 [==============================] - 2s 0us/step


In [ ]:
dataset_dir = os.path.join(os.path.dirname(dataset), '.')

In [ ]:
os.listdir(dataset_dir)

['.config',
 'test',
 'aclImdb_v1.tar.gz',
 'aclImdb',
 'train',
 'stack_overflow_16k.tar.gz',
 'README.md',
 'sample_data']

In [ ]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['java', 'python', 'javascript', 'csharp']

aclImdb/train/pos и aclImdb/train/neg содержат множество текстовых файлов, каждый из которых представляет собой отдельный обзор фильма. Давайте взглянем на один из них.

In [ ]:
sample_file = os.path.join(train_dir, 'pos/1181_9.txt')
with open(sample_file) as f:
  print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


### Загрузите набор данных
Далее вы загрузите данные с диска и подготовите их в формат, подходящий для обучения. Для этого вы будете использовать полезную утилиту text_dataset_from_directory , которая ожидает следующую структуру каталогов.

```
main_directory/
...class_a/
......a_text_1.txt
......a_text_2.txt
...class_b/
......b_text_1.txt
......b_text_2.txt
```

Чтобы подготовить набор данных для бинарной классификации, вам понадобятся две папки на диске, соответствующие class_a и class_b . Это будут положительные и отрицательные отзывы о фильмах, которые можно найти в aclImdb/train/pos и aclImdb/train/neg . Поскольку набор данных IMDB содержит дополнительные папки, вы удалите их перед использованием этой утилиты.




In [ ]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

FileNotFoundError: ignored

Далее вы будете использовать утилиту text_dataset_from_directory для создания помеченного tf.data.Dataset . tf.data — мощный набор инструментов для работы с данными.

При проведении эксперимента по машинному обучению рекомендуется разделить набор данных на три части: обучение , проверка и тестирование .

Набор данных IMDB уже разделен на обучающий и тестовый, но в нем отсутствует проверочный набор. Давайте создадим проверочный набор, используя разделение обучающих данных 80:20, используя приведенный ниже аргумент validation_split .

In [ ]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


Как вы можете видеть выше, в папке для обучения есть 25 000 примеров, из которых вы будете использовать 80% (или 20 000) для обучения. Как вы скоро увидите, вы можете обучить модель, передав набор данных непосредственно в model.fit . Если вы новичок в tf.data , вы также можете перебрать набор данных и распечатать несколько примеров следующим образом.

In [ ]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Review b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default constru

Обратите внимание, что обзоры содержат необработанный текст (со знаками препинания и случайными HTML-тегами, такими как <br/> ). Вы покажете, как обращаться с ними в следующем разделе.

Метки — 0 или 1. Чтобы увидеть, какие из них соответствуют положительным и отрицательным отзывам о фильмах, вы можете проверить свойство class_names в наборе данных.


In [ ]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to csharp
Label 1 corresponds to java


Далее вы создадите набор данных для проверки и тестирования. Вы будете использовать оставшиеся 5000 отзывов из обучающего набора для проверки.



In [ ]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [ ]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'test', 
    batch_size=batch_size)

Found 8000 files belonging to 4 classes.


### Подготовьте набор данных для обучения
Далее вы будете стандартизировать, токенизировать и векторизовать данные, используя полезный слой tf.keras.layers.TextVectorization .

Стандартизация относится к предварительной обработке текста, обычно для удаления знаков препинания или элементов HTML для упрощения набора данных. Токенизация относится к разбиению строк на токены (например, разбиение предложения на отдельные слова путем разбиения по пробелам). Векторизация относится к преобразованию токенов в числа, чтобы их можно было передать в нейронную сеть. Все эти задачи могут быть выполнены с помощью этого слоя.

Как вы видели выше, обзоры содержат различные теги HTML, такие как <br /> . Эти теги не будут удалены стандартизатором по умолчанию в слое TextVectorization (который по умолчанию преобразует текст в нижний регистр и удаляет знаки препинания, но не удаляет HTML). Вы напишете пользовательскую функцию стандартизации для удаления HTML.

In [ ]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

Далее вы создадите слой TextVectorization . Вы будете использовать этот слой для стандартизации, токенизации и векторизации наших данных. Вы устанавливаете для параметра output_mode значение int , чтобы создавать уникальные целочисленные индексы для каждого токена.

Обратите внимание, что вы используете функцию разделения по умолчанию и пользовательскую функцию стандартизации, которую вы определили выше. Вы также определите некоторые константы для модели, такие как явная максимальная sequence_length , которая заставит слой дополнять или усекать последовательности точно до значений sequence_length .

In [ ]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

Затем вы вызовете adapt , чтобы подогнать состояние слоя предварительной обработки к набору данных. Это заставит модель построить индекс строк для целых чисел.

In [ ]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [ ]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [ ]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'"set blank to quit on exception? i\'m using blank 3..i\'ve been looking around for an answer to this, but i haven\'t found it yet. basically, i\'m running several blank scripts into a game engine, and each script has its own entry point...i\'d rather not add try: except blocks through all of my code, so i was wondering if it\'s at all possible to tell blank to quit (or perhaps assign a custom function to that ""callback"") on finding its first error, regardless of where or what it found? ..currently, the game engine will continue after finding and hitting an error, making it more difficult than necessary to diagnose issues since running into one error may make a subsequent script not work (as it relies on variables that the error-ing script set, for example). any ideas? ..i know that i could redirect the console to a file to allow for easier scrolling, but just capturing the first error and stopping the game prematurely would be really useful...okay, a couple of extr

Как вы можете видеть выше, каждый токен был заменен целым числом. Вы можете найти токен (строку), которому соответствует каждое целое число, вызвав .get_vocabulary() на слое.

In [ ]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1287 --->  scissors
 313 --->  source
Vocabulary size: 10000


Вы почти готовы обучить свою модель. В качестве последнего шага предварительной обработки вы примените созданный ранее слой TextVectorization к набору данных для обучения, проверки и тестирования.

In [ ]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

### Создайте модель

Пришло время создать вашу нейронную сеть:

In [ ]:
embedding_dim = 16

In [ ]:
model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160016    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 160,033
Trainable params: 160,033
Non-trainable params: 0
__________________________________________________

Слои укладываются последовательно для построения классификатора:

Первый слой — это слой Embedding . Этот слой принимает обзоры в целочисленном кодировании и ищет вектор встраивания для каждого индекса слова. Эти векторы изучаются по мере обучения модели. Векторы добавляют измерение к выходному массиву. Результирующие размеры: (batch, sequence, embedding) . Чтобы узнать больше о встраиваниях, см. руководство по встраиванию слов .
Затем слой GlobalAveragePooling1D возвращает выходной вектор фиксированной длины для каждого примера путем усреднения по измерению последовательности. Это позволяет модели обрабатывать ввод переменной длины самым простым способом.
Этот выходной вектор фиксированной длины передается через полносвязный ( Dense ) слой с 16 скрытыми единицами.
Последний слой плотно связан с одним выходным узлом.



### Функция потерь и оптимизатор
Модель нуждается в функции потерь и оптимизаторе для обучения. Поскольку это проблема бинарной классификации, а модель выводит вероятность (одноэлементный слой с сигмовидной активацией), вы будете использовать функцию потерь losses.BinaryCrossentropy .

Теперь настройте модель для использования оптимизатора и функции потерь

In [ ]:
model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

### Обучите модель

In [ ]:
epochs = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/10
200/200 [==============================] - 6s 11ms/step - loss: 0.0591 - binary_accuracy: 0.2473 - val_loss: -0.7489 - val_binary_accuracy: 0.2587
Epoch 2/10
200/200 [==============================] - 2s 10ms/step - loss: -1.5439 - binary_accuracy: 0.2478 - val_loss: -2.6381 - val_binary_accuracy: 0.2587
Epoch 3/10
200/200 [==============================] - 2s 11ms/step - loss: -3.7099 - binary_accuracy: 0.2478 - val_loss: -5.3954 - val_binary_accuracy: 0.2587
Epoch 4/10
200/200 [==============================] - 2s 11ms/step - loss: -6.8418 - binary_accuracy: 0.2478 - val_loss: -9.2752 - val_binary_accuracy: 0.2587
Epoch 5/10
200/200 [==============================] - 2s 11ms/step - loss: -11.0657 - binary_accuracy: 0.2478 - val_loss: -14.2729 - val_binary_accuracy: 0.2587
Epoch 6/10
200/200 [==============================] - 2s 11ms/step - loss: -16.1969 - binary_accuracy: 0.2478 - val_loss: -20.2807 - val_binary_accuracy: 0.2587
Epoch 7/10
200/200 [=======================

### Оценить модель
Посмотрим, как поведет себя модель. Будет возвращено два значения. Потеря (число, которое представляет нашу ошибку, чем меньше значение, тем лучше) и точность.

In [ ]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

  1/782 [..............................] - ETA: 46s - loss: 0.2169 - binary_accuracy: 0.9062

 26/782 [..............................] - ETA: 1s - loss: 0.3031 - binary_accuracy: 0.8750 

 51/782 [>.............................] - ETA: 1s - loss: 0.3293 - binary_accuracy: 0.8695

 75/782 [=>............................] - ETA: 1s - loss: 0.3354 - binary_accuracy: 0.8621

 99/782 [==>...........................] - ETA: 1s - loss: 0.3264 - binary_accuracy: 0.8617

...

726/782 [==========================>...] - ETA: 0s - loss: 0.3098 - binary_accuracy: 0.8722

751/782 [===========================>..] - ETA: 0s - loss: 0.3105 - binary_accuracy: 0.8724

776/782 [============================>.] - ETA: 0s - loss: 0.3108 - binary_accuracy: 0.8728

782/782 [==============================] - 2s 2ms/step - loss: 0.3107 - binary_accuracy: 0.8727


Loss:  0.31072962284088135
Accuracy:  0.8726800084114075


Этот подход обеспечивает точность около 86%.

### Проверка на новых данных



In [ ]:
examples = [
  "The movie was great!",
  "The movie was okay.",
  "The movie was terrible..."
]

export_model.predict(examples)

array([[0.5921171 ],
       [0.41369876],
       [0.33293992]], dtype=float32)

## Упражнение: многоклассовая классификация по вопросам Stack Overflow
В этом руководстве показано, как с нуля обучить двоичный классификатор на наборе данных IMDB. Вы должны изменить эту записную книжку, чтобы обучить многоклассовый классификатор прогнозировать тег вопроса по программированию в Stack Overflow .

Для вас подготовлен набор данных , содержащий несколько тысяч вопросов по программированию **(например, «Как мне отсортировать словарь по значению в Python?»)**, размещенных в Stack Overflow. Каждый из них помечен ровно одним тегом (Python, CSharp, JavaScript или Java). Ваша задача — принять вопрос в качестве входных данных и предсказать соответствующий тег, в данном случае Python.

Набор данных, с которым вы будете работать, содержит несколько тысяч вопросов, извлеченных из гораздо более крупного общедоступного набора данных Stack Overflow в BigQuery , который содержит более 17 миллионов сообщений.

После загрузки набора данных вы обнаружите, что структура каталогов аналогична набору данных IMDB, с которым вы работали ранее:



```
train/
...python/
......0.txt
......1.txt
...javascript/
......0.txt
......1.txt
...csharp/
......0.txt
......1.txt
...java/
......0.txt
......1.txt
```



Чтобы выполнить это упражнение, вы должны изменить эту записную книжку для работы с набором данных Stack Overflow, внеся следующие изменения:

1. В верхней части записной книжки обновите код, который загружает набор данных IMDB, на код для загрузки уже подготовленного набора данных Stack Overflow . Поскольку набор данных Stack Overflow имеет аналогичную структуру каталогов, вам не потребуется вносить много изменений.

2. Измените последний слой вашей модели на Dense(4) , так как теперь есть четыре выходных класса.

3. При компиляции модели измените потери на tf.keras.losses.SparseCategoricalCrossentropy . Это правильная функция потерь для использования в задаче классификации нескольких классов, когда метки для каждого класса являются целыми числами (в данном случае они могут быть 0, 1 , 2 или 3 ). Кроме того, измените метрику на metrics=['accuracy'] , так как это проблема классификации нескольких классов ( tf.metrics.BinaryAccuracy используется только для бинарных классификаторов).

4. Как только эти изменения будут завершены, вы сможете обучить модель.

5. Протестируйте Вашу модель на следующем примере "How can I read a text file in Python?"

Подсказка по пункту 1: достаточно заменить только значение параметра url на адрес набора данных StackOverflow :
`"https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"`

In [ ]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"

dataset = tf.keras.utils.get_file("stack_overflow_16k", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), '.')

6062080/6053168 [==============================] - 0s 0us/step
